In [ ]:
import os

import numpy
import pandas
from scipy import stats
from statsmodels.stats import multitest


import capblood_seq
from capblood_seq import config

In [ ]:
GENE_ABUNDANCE_FILTER = 0.1

# Whether to pool subjects into one t-test (True) or perform a test on each subject
# separately and then combine via Stouffer's method (False)
POOL_SUBJECTS = False

# Whether to normalize means within each subject - recommend doing this
# if POOL_SUBJECTS is True
NORMALIZE_WITHIN_SUBJECT = False

In [ ]:
# Load the dataset. This downloads it if it doesn't exist already, and loads it into memory
dataset = capblood_seq.load_dataset(data_directory="data", pipeline_name="normalized")

In [ ]:
# We want to compare percentages only relative to the cells we were able to identify
dataset.filter_multi_labeled_cells(config.CELL_TYPES)
dataset.filter_genes_by_percent_abundance(GENE_ABUNDANCE_FILTER, any_sample=True)

In [ ]:
num_genes = dataset.get_num_genes()
num_cell_types = len(config.CELL_TYPES)
print("%s genes after filtering" % num_genes)

In [ ]:
# Preload transcript counts into numpy arrays for faster access

cell_type_subject_sample_transcript_counts = {}

for cell_type_index, cell_type in enumerate(config.CELL_TYPES + [None]):
    for subject_index, subject_id in enumerate(config.SUBJECT_IDS):
        for sample in config.SAMPLE_NAMES:
            
            transcript_counts = dataset.get_transcript_counts(
                sample,
                cell_type,
                subject_id,
                normalized=True
            )
            
            if transcript_counts is None:
                continue
            
            transcript_counts = transcript_counts.to_array()
            
            cell_type_subject_sample_transcript_counts[(cell_type, subject_id, sample)] = transcript_counts

In [ ]:
hypothesis_p_values = numpy.zeros((num_genes, num_cell_types + 1))
hypothesis_z_scores = numpy.zeros((num_genes, num_cell_types + 1))
num_cells_expressing_gene = numpy.zeros((num_genes, num_cell_types + 1))
num_cells = numpy.zeros((num_genes, num_cell_types + 1))

for cell_type_index, cell_type in enumerate(config.CELL_TYPES + [None]):
    
    for gene_index, gene in enumerate(dataset._gene_list):

        if gene_index % 1000 == 0:
            print(gene_index)
            
        subject_p_values = []
        num_samples_per_subject = []
        
        AM_means = []
        PM_means = []

        for subject_index, subject_id in enumerate(config.SUBJECT_IDS):
            
            subject_AM_means = []
            subject_PM_means = []
            
            for sample in config.SAMPLE_NAMES:
                
                if (cell_type, subject_id, sample) not in cell_type_subject_sample_transcript_counts:
                    continue
                
                transcript_counts = \
                    cell_type_subject_sample_transcript_counts[(cell_type, subject_id, sample)][:, gene_index]
                
                gene_mean = transcript_counts.mean()
                num_cells_expressing_gene[gene_index, cell_type_index] += \
                    transcript_counts[transcript_counts > 0].shape[0]
                num_cells[gene_index, cell_type_index] += transcript_counts.shape[0]
                
                if "AM" in sample:
                    subject_AM_means.append(gene_mean)
                else:
                    subject_PM_means.append(gene_mean)
            
            subject_AM_means = numpy.array(subject_AM_means)
            subject_PM_means = numpy.array(subject_PM_means)
            
            num_samples = len(subject_AM_means) + len(subject_PM_means)
            
            if NORMALIZE_WITHIN_SUBJECT:
                mean_of_means = \
                    (subject_AM_means.mean() * len(subject_PM_means) + \
                    subject_PM_means.mean() * len(subject_AM_means))/num_samples
                subject_AM_means -= mean_of_means
                subject_PM_means -= mean_of_means
            
            if not POOL_SUBJECTS:
                z, p = stats.ttest_ind(subject_AM_means, subject_PM_means)
                if numpy.isnan(z):
                    continue
                num_samples_per_subject.append(num_samples)
                subject_p_values.append(p)
            else:
                AM_means.extend(subject_AM_means)
                PM_means.extend(subject_PM_means)

        if not POOL_SUBJECTS:
            
            if len(subject_p_values) != 0:
                z, p = stats.combine_pvalues(subject_p_values, method="stouffer", weights=num_samples_per_subject)
            else:
                z = numpy.nan
                p = numpy.nan
        else:
            z, p = stats.ttest_ind(AM_means, PM_means)
        
        hypothesis_p_values[gene_index, cell_type_index] = p
        hypothesis_z_scores[gene_index, cell_type_index] = z

In [ ]:
hypothesis_cell_type_p_values = hypothesis_p_values[:, 0:num_cell_types]

diurnal_genes_df = \
    pandas.DataFrame(
        numpy.concatenate(
            (
                hypothesis_z_scores[:, 0:num_cell_types],
                numpy.array(
                    [hypothesis_z_scores[i, j] for i, j in enumerate(hypothesis_cell_type_p_values.argmin(axis=1))]
                ).reshape((-1, 1)),
                hypothesis_p_values[:, 0:num_cell_types],
                numpy.array(
                    [hypothesis_p_values[i, j] for i, j in enumerate(hypothesis_cell_type_p_values.argmin(axis=1))]
                ).reshape((-1, 1))
            ),
            axis=1
        ),
        index=dataset._gene_list,
        columns=[cell_type + " Z Score" for cell_type in config.CELL_TYPES + ["By Cell Type"]] + \
            [cell_type + " p-value" for cell_type in config.CELL_TYPES + ["By Cell Type"]]
    )

In [ ]:
diurnal_genes_df["Max Cell Type"] = [config.CELL_TYPES[i] for i in hypothesis_cell_type_p_values.argmin(axis=1)]

In [ ]:
# Get just the p values associated with all cells
hypothesis_all_cells_p_values = hypothesis_p_values[:, -1]

# Get the associated z score for the lowest p-value
all_cells_z_scores = hypothesis_z_scores[:, -1]

In [ ]:
diurnal_genes_df["Population Wide Z Score"] = all_cells_z_scores
diurnal_genes_df["Population Wide p-value"] = hypothesis_all_cells_p_values

In [ ]:
percent_cells_expressing_gene = (num_cells_expressing_gene/num_cells).max(axis=1)

In [ ]:
diurnal_genes_df["Percent Cells Expressing"] = percent_cells_expressing_gene

In [ ]:
diurnal_genes_df.to_csv(os.path.join("data", "gene_diurnality_scores.csv"))